##### ChromaDB

In [2]:
import os
from dotenv import load_dotenv

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document
import numpy as np

###### Vector store

In [20]:
from langchain_community.vectorstores import Chroma

In [21]:
### Create sample documents
sample_docs = [
'''
Machine Learning (ML)

Machine Learning is a subfield of Artificial Intelligence that focuses on building algorithms capable of learning patterns from data.
Instead of being explicitly programmed to perform tasks, ML systems improve automatically through experience.
Common ML techniques include supervised learning, where the system is trained with labeled data; unsupervised learning,
which finds hidden structures in unlabeled datasets; and reinforcement learning,
where agents learn optimal behavior by receiving rewards or penalties.
''',
'''
Deep Learning (DL)

Deep Learning is a specialized branch of Machine Learning that uses artificial neural networks with many layers to process complex data.
It has gained massive popularity in tasks like image classification, speech recognition,
and natural language understanding. Convolutional Neural Networks (CNNs) are widely used in computer vision,
while Recurrent Neural Networks (RNNs) and Transformers are more common in sequential data tasks.
Deep Learning often requires large datasets and high computational power but delivers state-of-the-art performance in many real-world applications.

''',
'''
Natural Language Processing (NLP)

Natural Language Processing is an area of AI that enables machines to understand, interpret,
and generate human language. It involves tasks like sentiment analysis, machine translation, text summarization,
and question answering. Traditional NLP relied heavily on statistical models and feature engineering,
but modern approaches are dominated by deep learning models such as BERT, GPT, and other transformer-based architectures.
NLP powers real-world systems like chatbots, voice assistants, and recommendation engines.
''',
'''
Bridging ML, DL, and NLP

While ML provides the fundamental building blocks for predictive modeling, Deep Learning takes these ideas further by
automatically learning hierarchical representations from data. NLP, on the other hand,
applies both ML and DL methods to language-related tasks. Together, these fields are revolutionizing
industries from healthcare and finance to education and entertainment, enabling smarter and more interactive AI systems.
'''
]

In [22]:
sample_docs

['\nMachine Learning (ML)\n\nMachine Learning is a subfield of Artificial Intelligence that focuses on building algorithms capable of learning patterns from data.\nInstead of being explicitly programmed to perform tasks, ML systems improve automatically through experience.\nCommon ML techniques include supervised learning, where the system is trained with labeled data; unsupervised learning,\nwhich finds hidden structures in unlabeled datasets; and reinforcement learning,\nwhere agents learn optimal behavior by receiving rewards or penalties.\n',
 '\nDeep Learning (DL)\n\nDeep Learning is a specialized branch of Machine Learning that uses artificial neural networks with many layers to process complex data.\nIt has gained massive popularity in tasks like image classification, speech recognition,\nand natural language understanding. Convolutional Neural Networks (CNNs) are widely used in computer vision,\nwhile Recurrent Neural Networks (RNNs) and Transformers are more common in sequenti

In [23]:
# save the sample documents to files
import tempfile
temp_dir = tempfile.mkdtemp()

for i,doc in enumerate(sample_docs):
    with open(f'{temp_dir}/doc_{i}.txt','w') as f:
        f.write(doc)

print(f'Sample document created in {temp_dir}')

Sample document created in /tmp/tmpjl6s185h


###### Document Loading

In [24]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader(
    temp_dir,
    glob='*.txt',
    loader_cls=TextLoader,
    loader_kwargs={'encoding':'utf-8'},

) 

In [25]:
documents = loader.load()

print(f'Loaded {len(documents)} document')
print(f'\nFirst document preview:-')
print(f'{documents[0].page_content}')

Loaded 4 document

First document preview:-

Bridging ML, DL, and NLP

While ML provides the fundamental building blocks for predictive modeling, Deep Learning takes these ideas further by
automatically learning hierarchical representations from data. NLP, on the other hand,
applies both ML and DL methods to language-related tasks. Together, these fields are revolutionizing
industries from healthcare and finance to education and entertainment, enabling smarter and more interactive AI systems.



###### Document Splitting

In [26]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
    length_function = len,
    separators=['\n\n','\n','. ','',' ']
)
chunks = text_splitter.split_documents(documents)

In [27]:
print(f'Created {len(chunks)} chunks from ({len(chunks)}) documents')
print(f'\nChunk example')
print(f'Content: {chunks[0].page_content}...')
print(f'Metadata: {chunks[0].metadata}')


Created 10 chunks from (10) documents

Chunk example
Content: Bridging ML, DL, and NLP

While ML provides the fundamental building blocks for predictive modeling, Deep Learning takes these ideas further by
automatically learning hierarchical representations from data. NLP, on the other hand,
applies both ML and DL methods to language-related tasks. Together, these fields are revolutionizing
industries from healthcare and finance to education and entertainment, enabling smarter and more interactive AI systems....
Metadata: {'source': '/tmp/tmpjl6s185h/doc_3.txt'}


In [28]:
chunks

[Document(metadata={'source': '/tmp/tmpjl6s185h/doc_3.txt'}, page_content='Bridging ML, DL, and NLP\n\nWhile ML provides the fundamental building blocks for predictive modeling, Deep Learning takes these ideas further by\nautomatically learning hierarchical representations from data. NLP, on the other hand,\napplies both ML and DL methods to language-related tasks. Together, these fields are revolutionizing\nindustries from healthcare and finance to education and entertainment, enabling smarter and more interactive AI systems.'),
 Document(metadata={'source': '/tmp/tmpjl6s185h/doc_2.txt'}, page_content='Natural Language Processing (NLP)'),
 Document(metadata={'source': '/tmp/tmpjl6s185h/doc_2.txt'}, page_content='Natural Language Processing is an area of AI that enables machines to understand, interpret,\nand generate human language. It involves tasks like sentiment analysis, machine translation, text summarization,\nand question answering. Traditional NLP relied heavily on statistical

###### Embedding Models

In [29]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [ ]:
sample_text = 'Machine Learning is fascinating'
embedding_model = OpenAIEmbeddings(
    model=''
)

###### Vector-store 

In [34]:
# create a Chromadb vector store
persistent_directory = "../chroma_db"

## Intialize Chromadb with OpenAI embeddings
vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=OpenAIEmbeddings(),
    persist_directory=persistent_directory,
    collection_name = 'rag_collection'
)

print(f'Vector store created with {vector_store._collection.count()} vectors')
print(f'Persisted to: {persistent_directory}')

Vector store created with 10 vectors
Persisted to: ../chroma_db


###### Test Similarity search

In [ ]:
query = "what are the tasks of NLP?"
similar_docs = vector_store.similarity_search(query,k=3)
similar_docs

[Document(metadata={'source': '/tmp/tmpjl6s185h/doc_2.txt'}, page_content='Natural Language Processing (NLP)'),
 Document(metadata={'source': '/tmp/tmpjl6s185h/doc_2.txt'}, page_content='NLP powers real-world systems like chatbots, voice assistants, and recommendation engines.'),
 Document(metadata={'source': '/tmp/tmpjl6s185h/doc_2.txt'}, page_content='Natural Language Processing is an area of AI that enables machines to understand, interpret,\nand generate human language. It involves tasks like sentiment analysis, machine translation, text summarization,\nand question answering. Traditional NLP relied heavily on statistical models and feature engineering,\nbut modern approaches are dominated by deep learning models such as BERT, GPT, and other transformer-based architectures.')]

In [43]:
query = "What is the Deep Learning?"
similar_docs = vector_store.similarity_search(query,k=3)
similar_docs

[Document(metadata={'source': '/tmp/tmpjl6s185h/doc_1.txt'}, page_content='Deep Learning (DL)'),
 Document(metadata={'source': '/tmp/tmpjl6s185h/doc_1.txt'}, page_content='Deep Learning is a specialized branch of Machine Learning that uses artificial neural networks with many layers to process complex data.\nIt has gained massive popularity in tasks like image classification, speech recognition,\nand natural language understanding. Convolutional Neural Networks (CNNs) are widely used in computer vision,\nwhile Recurrent Neural Networks (RNNs) and Transformers are more common in sequential data tasks.'),
 Document(metadata={'source': '/tmp/tmpjl6s185h/doc_1.txt'}, page_content='Deep Learning often requires large datasets and high computational power but delivers state-of-the-art performance in many real-world applications.')]

In [37]:
query = "What is NLP?"
similar_docs = vector_store.similarity_search(query,k=3)
similar_docs

[Document(metadata={'source': '/tmp/tmpjl6s185h/doc_2.txt'}, page_content='Natural Language Processing (NLP)'),
 Document(metadata={'source': '/tmp/tmpjl6s185h/doc_2.txt'}, page_content='Natural Language Processing is an area of AI that enables machines to understand, interpret,\nand generate human language. It involves tasks like sentiment analysis, machine translation, text summarization,\nand question answering. Traditional NLP relied heavily on statistical models and feature engineering,\nbut modern approaches are dominated by deep learning models such as BERT, GPT, and other transformer-based architectures.'),
 Document(metadata={'source': '/tmp/tmpjl6s185h/doc_2.txt'}, page_content='NLP powers real-world systems like chatbots, voice assistants, and recommendation engines.')]

In [44]:
print(f'Query: {query}')
print(f'{len(similar_docs)} similar chunks')
for i,doc in enumerate(similar_docs):
    print(f'---- Chunk {i+1} ----')
    print(doc.page_content[:200] + '...')
    print(f'Source: {doc.metadata.get('source','unknown')}')

Query: What is the Deep Learning?
3 similar chunks
---- Chunk 1 ----
Deep Learning (DL)...
Source: /tmp/tmpjl6s185h/doc_1.txt
---- Chunk 2 ----
Deep Learning is a specialized branch of Machine Learning that uses artificial neural networks with many layers to process complex data.
It has gained massive popularity in tasks like image classifica...
Source: /tmp/tmpjl6s185h/doc_1.txt
---- Chunk 3 ----
Deep Learning often requires large datasets and high computational power but delivers state-of-the-art performance in many real-world applications....
Source: /tmp/tmpjl6s185h/doc_1.txt


In [46]:
scores = vector_store.similarity_search_with_score(query,k=3)
print(scores)

[(Document(metadata={'source': '/tmp/tmpjl6s185h/doc_1.txt'}, page_content='Deep Learning (DL)'), 0.1618431955575943), (Document(metadata={'source': '/tmp/tmpjl6s185h/doc_1.txt'}, page_content='Deep Learning is a specialized branch of Machine Learning that uses artificial neural networks with many layers to process complex data.\nIt has gained massive popularity in tasks like image classification, speech recognition,\nand natural language understanding. Convolutional Neural Networks (CNNs) are widely used in computer vision,\nwhile Recurrent Neural Networks (RNNs) and Transformers are more common in sequential data tasks.'), 0.20138491690158844), (Document(metadata={'source': '/tmp/tmpjl6s185h/doc_1.txt'}, page_content='Deep Learning often requires large datasets and high computational power but delivers state-of-the-art performance in many real-world applications.'), 0.2461816966533661)]


##### Initialize LLM, RAG Chain,Prompt Template,Query the RAG system

In [48]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name = 'gpt-4o-mini',

)

In [50]:
test_response = llm.invoke('What is LLM?')
test_response

AIMessage(content='LLM stands for "Large Language Model." It refers to a type of artificial intelligence model that is trained on vast amounts of text data to understand and generate human-like language. These models use deep learning techniques, particularly neural networks, to learn patterns, grammar, facts, and some level of reasoning from the text.\n\nKey characteristics of LLMs include:\n\n1. **Scale**: LLMs are characterized by their large number of parameters, often in the billions or trillions, which allows them to capture a wide range of linguistic nuances and knowledge.\n\n2. **Versatility**: They can perform numerous language-related tasks, including but not limited to text generation, translation, summarization, question answering, and conversational agents.\n\n3. **Pre-training and Fine-tuning**: They are typically pre-trained on a broad dataset in an unsupervised manner and can be fine-tuned on specific tasks with smaller, supervised datasets.\n\n4. **Applications**: LLMs

In [51]:
from langchain.chat_models.base import init_chat_model

llm = init_chat_model('openai:gpt-4o-mini')
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7f87eea14a50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7f87eea14b90>, root_client=<openai.OpenAI object at 0x7f87eea90fc0>, root_async_client=<openai.AsyncOpenAI object at 0x7f87eea90e90>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [52]:
llm.invoke('What is AI?')

AIMessage(content='Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are designed to think and act like humans. This encompasses a variety of capabilities, including:\n\n1. **Machine Learning**: Systems that can learn from data, identify patterns, and make decisions with minimal human intervention.\n  \n2. **Natural Language Processing (NLP)**: The ability of machines to understand and interpret human language, enabling applications such as voice assistants and chatbots.\n\n3. **Computer Vision**: The capability of machines to interpret and understand visual information from the world, allowing for tasks like image recognition and autonomous driving.\n\n4. **Robotics**: The creation of machines that can perform tasks traditionally done by humans, often incorporating aspects of AI for navigation and decision-making.\n\n5. **Expert Systems**: AI systems that mimic the decision-making abilities of a human expert in specific fields.\n\nAI can be c

##### Modern RAG Chain

In [53]:
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

In [54]:
### convert vector store to retriever
retriever = vector_store.as_retriever(
    search_kwargs={'k':3}

)
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f8836447230>, search_kwargs={'k': 3})

In [57]:
## create a prompt template
system_prompt = """
You are an assistant for question-answering tasks.Use the following pieces of retrieved context to answer the question.
if you dont know the answer,just say that you dont know.
use three sentences maximum and keep the answer concise.

Context: {context}"""

prompt = ChatPromptTemplate.from_messages([
    ('system',system_prompt),
    ('human','{input}')
])

In [58]:
prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nYou are an assistant for question-answering tasks.Use the following pieces of retrieved context to answer the question.\nif you dont know the answer,just say that you dont know.\nuse three sentences maximum and keep the answer concise.\n\nContext: {context}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [59]:
### Create a document chain
document_chain = create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nYou are an assistant for question-answering tasks.Use the following pieces of retrieved context to answer the question.\nif you dont know the answer,just say that you dont know.\nuse three sentences maximum and keep the answer concise.\n\nContext: {context}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7f87eea14a50>, async_client=<openai.resources.cha

This chain:
- takes retrieved documents
- 'Stuffs' them into the prompts {context} placeholder
- sends the complete prompt to the LLM
- Returns the LLM's response

In [60]:
### Create the final RAG Chain
rag_chain = create_retrieval_chain(retriever,document_chain)
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f8836447230>, search_kwargs={'k': 3}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nYou are an assistant for question-answering tasks.Use the following pieces of retrieved context to answer the question.\nif you dont

In [62]:
response=rag_chain.invoke({'input':'Hey,What are the tasks of NLP?'})


In [63]:
response['answer']

'NLP involves tasks such as sentiment analysis, machine translation, text summarization, and question answering. These tasks help machines understand, interpret, and generate human language.'

In [64]:
# Function to query the modern RAG system
def query_rag_modern(question):
    print(f'Questions: {question}')
    print("-"*30)

    result = rag_chain.invoke({'input':question})

    print(f'Answer: {result['answer']}')
    print(f'\nRetrieved Context')
    for i,doc in enumerate(result['context']):
        print(f'\n--- Source {i+1} ---')
        print(doc.page_content[:200] + '---')
    return result

test_question = [
    'What is the Machine Learning?',
    'What is the use of CNNs',
    'Full form of NLP'
]

for question in test_question:
    result = query_rag_modern(question)
    print(f'\n'+ '='*80 + '\n')

Questions: What is the Machine Learning?
------------------------------
Answer: Machine Learning is a subfield of Artificial Intelligence that focuses on building algorithms that learn patterns from data. It enables systems to improve automatically through experience instead of being explicitly programmed for tasks. Common techniques include supervised, unsupervised, and reinforcement learning.

Retrieved Context

--- Source 1 ---
Machine Learning (ML)---

--- Source 2 ---
Machine Learning is a subfield of Artificial Intelligence that focuses on building algorithms capable of learning patterns from data.
Instead of being explicitly programmed to perform tasks, ML system---

--- Source 3 ---
Deep Learning (DL)---


Questions: What is the use of CNNs
------------------------------
Answer: Convolutional Neural Networks (CNNs) are primarily used in computer vision tasks such as image classification, object detection, and image segmentation. They excel at processing and analyzing visual dat

#### Create RAG Chain Alternative - Using LCEL (LangChain Expression Language)

In [65]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableParallel

In [68]:
from langchain_core.prompts import ChatPromptTemplate

custom_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful, knowledgeable assistant.
- Always use the provided context to answer questions. 
- If the context does not have the answer, say "I don't know."
- Be clear and concise in explanations.
- Do not make assumptions beyond the given context.
- Stay on-topic and relevant to the user’s query."""),

    ("user", """Context:
{context}

Question: {question}

Answer:""")
])


In [69]:
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f8836447230>, search_kwargs={'k': 3})

In [70]:
### Formate the output documents for the prompt
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
## Build the chain using LCEL

rag_chain_lcel = (
    {
    'context':retriever | format_docs,
     "question":RunnablePassthrough()
     }
    | custom_prompt
    | llm 
    | StrOutputParser()
)

In [72]:
rag_chain_lcel

{
  context: VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f8836447230>, search_kwargs={'k': 3})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful, knowledgeable assistant.\n- Always use the provided context to answer questions. \n- If the context does not have the answer, say "I don\'t know."\n- Be clear and concise in explanations.\n- Do not make assumptions beyond the given context.\n- Stay on-topic and relevant to the user’s query.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Context:\n{context}\n\nQuestion: {que

In [73]:
response=rag_chain_lcel.invoke('What is the Machine Learning?')
response

'Machine Learning is a subfield of Artificial Intelligence that focuses on building algorithms capable of learning patterns from data. Instead of being explicitly programmed to perform tasks, ML systems improve automatically through experience. Common techniques include supervised learning, unsupervised learning, and reinforcement learning.'

In [74]:
retriever.get_relevant_documents('What is the Machine Learning?')

/tmp/ipykernel_39736/4172794280.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents('What is the Machine Learning?')


[Document(metadata={'source': '/tmp/tmpjl6s185h/doc_0.txt'}, page_content='Machine Learning (ML)'),
 Document(metadata={'source': '/tmp/tmpjl6s185h/doc_0.txt'}, page_content='Machine Learning is a subfield of Artificial Intelligence that focuses on building algorithms capable of learning patterns from data.\nInstead of being explicitly programmed to perform tasks, ML systems improve automatically through experience.\nCommon ML techniques include supervised learning, where the system is trained with labeled data; unsupervised learning,\nwhich finds hidden structures in unlabeled datasets; and reinforcement learning,'),
 Document(metadata={'source': '/tmp/tmpjl6s185h/doc_1.txt'}, page_content='Deep Learning (DL)')]

In [75]:
def query_rag_lcel(question):
    print(f'Question: {question}')
    print('-' * 50)

    answer = rag_chain_lcel.invoke(question)
    print(f'Answer: {answer}')

    docs = retriever.get_relevant_documents(question)
    print("\nSource Documents:")
    for i,doc in enumerate(docs):
        print(f'\n--- Source {i+1} ---')
        print(doc.page_content[:200] + '...')

In [76]:
query_rag_lcel("What is the term 'DL'?")

Question: What is the term 'DL'?
--------------------------------------------------
Answer: 'DL' stands for Deep Learning, which is a subset of Machine Learning that focuses on automatically learning hierarchical representations from data.

Source Documents:

--- Source 1 ---
Deep Learning (DL)...

--- Source 2 ---
Bridging ML, DL, and NLP

While ML provides the fundamental building blocks for predictive modeling, Deep Learning takes these ideas further by
automatically learning hierarchical representations from...

--- Source 3 ---
Machine Learning (ML)...


In [ ]:
query_rag_lcel("What is the term 'DL'?")

##### Adding new documents to the Existing vector store

In [78]:
vector_store

In [82]:
new_documet = '''
Embeddings
Embeddings are dense vector representations of text that capture semantic meaning.
Unlike bag-of-words, embeddings place similar words close together in vector space.
Popular models:
Word2Vec and GloVe (classical).
BERT embeddings (contextual).
OpenAI embeddings (optimized for search + retrieval).
Retrieval-Augmented Generation (RAG)
RAG is an architecture that combines a retriever with a generator.
The retriever fetches relevant chunks from a vector store based on embeddings similarity.
The generator (LLM) conditions on both the query and retrieved docs to produce grounded answers.
Advantages:
Keeps responses factual.
Allows continuous knowledge updates without retraining the LLM.
'''

In [83]:
new_documet

'\nEmbeddings\nEmbeddings are dense vector representations of text that capture semantic meaning.\nUnlike bag-of-words, embeddings place similar words close together in vector space.\nPopular models:\nWord2Vec and GloVe (classical).\nBERT embeddings (contextual).\nOpenAI embeddings (optimized for search + retrieval).\nRetrieval-Augmented Generation (RAG)\nRAG is an architecture that combines a retriever with a generator.\nThe retriever fetches relevant chunks from a vector store based on embeddings similarity.\nThe generator (LLM) conditions on both the query and retrieved docs to produce grounded answers.\nAdvantages:\nKeeps responses factual.\nAllows continuous knowledge updates without retraining the LLM.\n'

In [84]:
new_doc = Document(
    page_content=new_documet,
    metadata = {'source':'manual_addition','topic':'embeddings'}
)

In [86]:
# split the document
new_chunks=text_splitter.split_documents([new_doc])
new_chunks

[Document(metadata={'source': 'manual_addition', 'topic': 'embeddings'}, page_content='Embeddings\nEmbeddings are dense vector representations of text that capture semantic meaning.\nUnlike bag-of-words, embeddings place similar words close together in vector space.\nPopular models:\nWord2Vec and GloVe (classical).\nBERT embeddings (contextual).\nOpenAI embeddings (optimized for search + retrieval).\nRetrieval-Augmented Generation (RAG)\nRAG is an architecture that combines a retriever with a generator.'),
 Document(metadata={'source': 'manual_addition', 'topic': 'embeddings'}, page_content='The retriever fetches relevant chunks from a vector store based on embeddings similarity.\nThe generator (LLM) conditions on both the query and retrieved docs to produce grounded answers.\nAdvantages:\nKeeps responses factual.\nAllows continuous knowledge updates without retraining the LLM.')]

In [85]:
new_doc

Document(metadata={'source': 'manual_addition', 'topic': 'embeddings'}, page_content='\nEmbeddings\nEmbeddings are dense vector representations of text that capture semantic meaning.\nUnlike bag-of-words, embeddings place similar words close together in vector space.\nPopular models:\nWord2Vec and GloVe (classical).\nBERT embeddings (contextual).\nOpenAI embeddings (optimized for search + retrieval).\nRetrieval-Augmented Generation (RAG)\nRAG is an architecture that combines a retriever with a generator.\nThe retriever fetches relevant chunks from a vector store based on embeddings similarity.\nThe generator (LLM) conditions on both the query and retrieved docs to produce grounded answers.\nAdvantages:\nKeeps responses factual.\nAllows continuous knowledge updates without retraining the LLM.\n')

In [87]:
vector_store.add_documents(new_chunks)

['83d3d240-c365-4fe6-b9dc-142676684f76',
 'bf90f595-c408-48ed-80d7-6d5bc304e293']

In [88]:
print(f'Total vectors now: {vector_store._collection.count()}')

Total vectors now: 12


In [89]:
#### Query with the updated vector
new_question = "what are all the embeddings models?"
result=query_rag_lcel(new_question)
result

Question: what are all the embeddings models?
--------------------------------------------------
Answer: The embeddings models mentioned are:

1. Word2Vec (classical)
2. GloVe (classical)
3. BERT embeddings (contextual)
4. OpenAI embeddings (optimized for search and retrieval)

Source Documents:

--- Source 1 ---
Embeddings
Embeddings are dense vector representations of text that capture semantic meaning.
Unlike bag-of-words, embeddings place similar words close together in vector space.
Popular models:
Word2V...

--- Source 2 ---
The retriever fetches relevant chunks from a vector store based on embeddings similarity.
The generator (LLM) conditions on both the query and retrieved docs to produce grounded answers.
Advantages:
K...

--- Source 3 ---
Bridging ML, DL, and NLP

While ML provides the fundamental building blocks for predictive modeling, Deep Learning takes these ideas further by
automatically learning hierarchical representations from...


#### Advanced RAG techniques 
- Previous Conversational Context

In [90]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage,AIMessage

In [104]:
from langchain.chains import create_history_aware_retriever
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder


contextualize_q_system_prompt = """Given a chat history and the latest user question which might reference context
in the chat history, formulate a standalone question which can be understood without the chat history.
Do not answer the question, just reformulate it if needed, otherwise return it as it is."""



contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])


In [105]:
history_aware_retriever = create_history_aware_retriever(
    llm,
    retriever,
   contextualize_q_prompt
)


In [107]:
qa_system_prompt = """you are an assistant for question-answering tasks. Use the following pieces of retrieved context
to answer the question. if you dont know the answer, just say that you dont know.
use three sentences maximum and keep the answer concise

Context: {context}"""



qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(llm,qa_prompt)

conversational_rag_chain = create_retrieval_chain(
    history_aware_retriever,
    question_answer_chain
)

print("Conversational RAG chain created!")

Conversational RAG chain created!


In [108]:
chat_history=[]
result = conversational_rag_chain.invoke({
    "chat_history":chat_history,
    "input":"What is machine learning?"
})
print(f"Q: What is machine learning?")
print(f"A : {result['answer']}")

Q: What is machine learning?
A : Machine Learning is a subfield of Artificial Intelligence that focuses on building algorithms capable of learning patterns from data. Instead of being explicitly programmed, ML systems improve automatically through experience. Common techniques include supervised learning, unsupervised learning, and reinforcement learning.


In [109]:
chat_history.extend([
    HumanMessage(content="What is machine learning"),
    AIMessage(content=result['answer'])
])

In [110]:
## Follow up question
result1 = conversational_rag_chain.invoke({
    "chat_history":chat_history,
    "input":"What are its main types?" # Refers to ML from previous question
})
result1

{'chat_history': [HumanMessage(content='What is machine learning', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Machine Learning is a subfield of Artificial Intelligence that focuses on building algorithms capable of learning patterns from data. Instead of being explicitly programmed, ML systems improve automatically through experience. Common techniques include supervised learning, unsupervised learning, and reinforcement learning.', additional_kwargs={}, response_metadata={})],
 'input': 'What are its main types?',
 'context': [Document(metadata={'source': '/tmp/tmpjl6s185h/doc_0.txt'}, page_content='Machine Learning (ML)'),
  Document(metadata={'source': '/tmp/tmpjl6s185h/doc_0.txt'}, page_content='Machine Learning is a subfield of Artificial Intelligence that focuses on building algorithms capable of learning patterns from data.\nInstead of being explicitly programmed to perform tasks, ML systems improve automatically through experience.\nCommon ML techniques i

In [111]:
result1['answer']

'The main types of Machine Learning are supervised learning, unsupervised learning, and reinforcement learning. Supervised learning involves training with labeled data, while unsupervised learning finds hidden structures in unlabeled datasets. Reinforcement learning is based on training agents to make decisions through rewards and penalties.'

##### Using GROQ LLM

In [4]:
load_dotenv()

True

In [6]:
from langchain_groq import ChatGroq
from langchain.chat_models import init_chat_model

In [7]:
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [8]:
llm = ChatGroq(model='gemma2-9b-it')
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f595c42bcb0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f595ee786e0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [9]:
llm = init_chat_model(model='groq:gemma2-9b-it')
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f595ee5d6d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f595ee5e0d0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))